# Análise de sentimento - Comentários de produtos eletrônicos do youtube - Armazenamento no Firebase

In [2]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from datetime import datetime
import pandas as pd
import time
import numpy as np

cred = credentials.Certificate('bkp/serviceAccountKey.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
comments_ref = db.collection(u'video-comments-count').document('count')
docs = comments_ref.get()

In [4]:
#print(f'Document data: {str(docs.to_dict())}')

#print(docs.to_dict()['video-comments-count'])
arr_ids = [] 
arr_counts = []

for comment in docs.to_dict()['video-comments-count'].split("|"):
    arr_ids.append(str(comment.split(",")[0]))
    arr_counts.append(int(comment.split(",")[1]))

dict_to_dataframe = {
    "comments_id": arr_ids,
    "comments_count": arr_counts
}

df = pd.DataFrame(dict_to_dataframe)

In [5]:
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

In [6]:
comments_report = {
    "data": dt_string,
    "comentarios avaliados": df['comments_count'].loc[df['comments_count'] != 100000].count(),
    "comentarios avaliados %":  df['comments_count'].loc[df['comments_count'] != 100000].count()/df['comments_count'].count()*100,
}

total_availables = 0
for values in np.delete(df['comments_count'].unique(),np.where(df['comments_count'].unique() == 100000)):
    comments_report["comentários com "+str(100000-values)]=df['comments_count'].loc[df['comments_count'] == values].count()
    total_availables = (100000-values) * df['comments_count'].loc[df['comments_count'] == values].count() + total_availables


comments_report['comentarios avaliados totais']=total_availables
comments_report = pd.DataFrame.from_records([comments_report])

comments_report

,data,comentarios avaliados,comentarios avaliados %,comentários com 2,comentários com 3,comentários com 1,comentários com 4,comentários com 5,comentários com 6,comentarios avaliados totais
0,06/09/2021 10:36:01,4908,49.08,1142,232,3491,35,7,1,6652


In [8]:
comments_report.to_csv("src/comments_site_analysis.csv")